# Study Notes - Lattice Boltzmann Method 

## Textbook:
"Lattice Boltzmann Method - Fundamentals and Engineering Applications with Computer Codes" by A. A. Mohamad

## Dependencies:
- Julia Version 1.8.0
- Jupyter Lab Version 3.4.5

In [1]:
include("Utils.jl")

┌ Warning: Pkg.installed() is deprecated
└ @ Pkg /cache/build/default-amdci4-3/julialang/julia-release-1-dot-8/usr/share/julia/stdlib/v1.8/Pkg/src/Pkg.jl:675
    Updating registry at `/opt/julia/registries/General.toml`
   Resolving package versions...
  No Changes to `/opt/julia/environments/v1.8/Project.toml`
  No Changes to `/opt/julia/environments/v1.8/Manifest.toml`


Main.Utils

# Lattice Boltzmann Method (LBM)

LBM bridged the gap between micro-scale (molecular dynamics simulation (MD/MDS) and macro-scale (Navier-Stokes (NS) Equation with finite element methods). The porperty of the collection of particles is represented by a distribution function which acts as a representative for collection of particles. Called meso-scale. 

## Advantages
* Easy to apply for complex domains
* Easy to treat multi-phase and multi-component flows without a need to trace the interfaces between different phases.
* It can be naturally adapted to parallel processes computing.
* There is no need to solve Laplace equation at each time step to satisfy continuity equation of incompressible, unsteady flows, as it is in solving NS equations.
* Handle problem in micro- and macro-scales with reliable accuracy.

## Disadvantages
* Needs more computer memory compared with NS solver.

## Particle Dynamics

Molecules moveing randomly in a free space with conservation of the mass, momentum, and energy.

### Newton's second law (momentum conservation)

rate of change of lienar momentum is equal to the net applied force. $$F = \frac{d(m\;c)}{dt}$$ F is the inter-molecular and external forces, m is the mass of the particle, c is the velocity vector of the particle and t is the time.

Assume constant mass, equation can be simplified as $F = m \frac{dc}{dt} = m\;a$ where a is the acceleration vector.

Position of particle can be determined from definition of velocity $c = \frac{dr}{dt}$ where r is the position vector of the particle relative to the origin.

MD simulation will solving above equation provided F is a known function.

If external force, F, is applied to a particle of mass m, the velocity will change from $c$ to $c + Fdt/m$ and position changes from $r$ to $r + cdt$.

## Pressure and Temperature

### Physics Notations
- $L$ is the length of the tube.
- $c_x$ is the speed of the particle in the x-direction.
- $c$ is the speed of the particle in all direction.
- $p$ is momentum
- $m$ is particle mass.
- $F$ is the force.
- $n$ is the number of molecules per unit volume
- $KE$ is kinetic energy.


### Physics formula

Consider a scenario where a particle is bouncing between two ends of a tube.

The time between hits (either end) is given by 
$$\Delta t = \frac{2L}{c_x}$$

Using the definition and formula of momentum, we can find that when the particle hits the wall and then returns, it has a momentum of 
$$F \cdot \Delta t = p = mv = mc_x - (-mc_x) = 2mc_x$$

Combining this with the assumption that the velocities in all three directions are equal, we have:
$$F \cdot \Delta t = F \cdot \left(\frac{2L}{c_x}\right) = 2mc_x$$

which implies
$$F = \frac{Nmc^2}{3L}$$

Pressure is defined as force per unit area perpendicular to the force. In general, $P = \frac{F}{A}$, where $A$ is the area.

Hence we have:
$$P = \frac{Nmc^2}{3LA} = \frac{Nmc^2}{3V} = \frac{1}{2}mc^2 \cdot \frac{2N}{3V} = \frac{2}{3}nKE$$


Combining this with the state equation for ideal gas
$$PV=nRT$$

and introducing the Boltzmann constant ($k_B$), we can conclude that **the relationship between pressure and temperature in ideal gases** is:
$$
P = \frac{2}{3}nKE = \frac{2}{3}n\frac{3}{2}k_BT = nk_BT 
$$

## Distribution Function f(c)

The idea of Maxwell is that the knowledge of velocity and position of each molecule at every instant of time is not important. Instead, we can use a distribution function to characterize the effect of molecules.

A distribution function takes a speed argument $c$ and returns the probability of particles moving at a speed around $c$.

For a gas of N particles, the number of particles having a velocity in the x-direction between $c_x$ and $c_x+dc_x$ is 
$$N \cdot f(c_x)$$

Since direction is relative to the observer, we want to remove the directional constraints and let the distribution function depend only on speed. In other words
$$f(c_x)f(c_y)f(c_z) = \Phi(c_x^2+c_y^2+c_z^2)$$

We square the speed to avoid neagtive value.

Additionally, this is a probability distribution function, which means we need to normalize it by setting 
$$\int\int\int f(c_x)f(c_y)f(c_z) dc_xdc_ydc_z = 1$$

There are two kinds of functions that satisfy this requirement: 
$$\begin{aligned}
    LogA + LogB &= LogAB\\
    e^Ae^B &= e^{(A+B)}
\end{aligned}$$

The latter is shown to be $\Phi$. Hence, we have: 
$$f(c) = A^3e^{(Bc^2)}$$

Now let's consider the formula in velocity space, where each particle can be thought of as a vector originating from the origin and pointing in a specific direction with the speed as its magnitude. Therefore, the number of particles with speeds between $c$ and $c+dc$ is equal to the number of points lying between two shells of a sphere with radii $c$ and $c+dc$.

The surface area of such a ring torus is $4\pi c^2dc$. Therefore, the probability distribution as a function of speed is given by:
$$f(c)dc = 4\pi c^2 A^3 e^{-Bc^2} dc$$
Here, $A$ and $B$ can be found by integrating $f(c)$ over all possible speeds to find the total number of particles $N$ and their total energy $E$.

Using the kinetic energy $\frac{1}{2}mc^2$ combined with the previous results, along with the normalizing constant, we have the **final result**:
$$
f(c) = 4\pi \left(\frac{m}{2\pi k_B T}\right)^\frac{3}{2} c^2 e^{-\frac{mc^2}{2k_BT}}
$$

## Maxwell-Boltzmann Distribution

Boltzmann generalized Maxwell's idea and realized that the number of uniform arrangements is far larger than the possibility of all gas particles staying, for example, in the left half of a box. Therefore, the probability of extreme cases is negligible.

**Theorem**: The thermodynamic entropy S of a system (at a given energy E) is related to the number W of microscopic states available to it by the equation:
$$S = k_B \log(W)$$

**Theorem**: For any system in thermal equilibrium at temperature T, the probability of being in a particular state at energy E is proportional to $e^{-\frac{E}{k_BT}}$, i.e.,
$$f(E) = A e^{-\frac{E}{k_BT}}$$

Considering the case in the x-direction, since $E = \frac{1}{2}mc_x^2$ and the normalizing constant $\int_{-\infty}^\infty Ae^{-\frac{mc_x^2}{2k_BT}} = 1$, it implies
$$A = \sqrt{\frac{m}{2\pi k_B T}}$$

This results in a probability of velocity $c_x$ given by
$$f(c_x) = \sqrt{\frac{m}{2\pi k_B T}} e^{-\frac{mc_x^2}{2k_BT}}$$

Multiplying $f(c_x)$ with the distribution function in the y and z directions gives:

$$
f(c) = \left(\frac{m}{2\pi k_B T}\right)^\frac{3}{2} e^{-\frac{mc^2}{2k_BT}}
$$

Multiplying $f(c)$ by $4\pi c^2$ (the surface area of a sphere in phase space) to account for the density of velocity states available to the particles gives the formula mentioned in the previous section.

**Conclusion**: An ideal gas follows a Maxwell-Boltzmann distribution function at equilibrium.

This leaves us with an interesting question of how to reach equilibrium assuming starting at non-equilibrium.

## Boltzmann Transport Equation

A **statistical description of a system** can be explained by a distribution function $f(r, c, t)$, which represents the number of molecules at time $t$ positioned between $r$ and $r + dr$, with velocities between $c$ and $c + dc$.

When an external force acts on gas molecules of unit mass, it changes the velocity of each molecule from $c$ to $c + F \cdot dt$ and its position from $r$ to $r + c \cdot dt$.

If no collision occurs, we have:
$$
f(r + c \cdot dt, c + F \cdot dt, t + dt) \, dr \, dc = f(r, c, t) \, dr \, dc
$$

However, in the case of a collision, there will be a net change in the number of molecules in the interval $dr \, dc$. Some particles may get accelerated, while others may slow down due to collisions. As a result, they will move to different speed intervals, leading to a net change in the distribution function.

**Define** the rate of change between final and initial status of the distribution function is called **collision operator** $\Omega$

Resulting in equation of number of molecules to be:
$$
f(r + c \cdot dt, c + F \cdot dt, t + dt) \, dr \, dc - f(r, c, t) \, dr \, dc = \Omega(f)
$$

Divide equation above by $dtdrdc$ and take $lim_{dt\rightarrow 0}$ yields
$$\frac{df}{dt} = \Omega(f)$$
(meaning: Total rate of change of the distribution function is equal to the rate of the collision)

By chain rule of multivariable function, we have
$$
\frac{df}{dt} = \frac{\partial f}{\partial r}\cdot \frac{dr}{dt} + \frac{\partial f}{\partial c}\cdot \frac{dc}{dt} + \frac{\partial f}{\partial t} = \frac{\partial f}{\partial r}\cdot c + \frac{\partial f}{\partial c}\cdot a + \frac{\partial f}{\partial t}
$$
where $a$ is acceleration and it has been related to force by $F=m\cdot a$

Hence, we finally have our **Boltzmann Transport Equation** (BTE)
$$
\frac{\partial f}{\partial t} + c\cdot\frac{\partial f}{\partial r} + \frac{F}{m}\cdot\frac{\partial f}{\partial c} = \Omega(f)
$$

In no external force case (similar to advection equation with source term):
$$
\frac{\partial f}{\partial t} + c\cdot \nabla f = \Omega(f)
$$

Relation to macroscopic quantities:
$$\begin{aligned}
\text{Fluid Density:}\; \rho(r, t) &= \int m f(r, c, t) dc \\
\text{Fluid Velocity vector:}\; \rho(r, t)u(r, t) &= \int m c f(r, c, t) dc \\
\text{Internal Energy:}\; \rho(r, t)e(r, t) &= \frac{1}{2}\int m u_a^2 f(r, c, t) dc = \frac{3}{2m}k_BT
\end{aligned}$$
where $u_a = c - u$ is particle velocity relative to the fluid velocity

## BGKW Approximation

Bhatnagar, Gross and Krook (BGK) in 1954 introduced a simplified model for collision operator. At the same time Welander (1954), independently, introduced similar operator.
$$\Omega = \omega (f^{eq} - f) = \frac{1}{\tau}(f^{eq}-f)$$
where $\omega$ is called the collision frequency and $\tau$ is called relaxation factor. $f^{eq}$ is the local equilibrium distribution function which is
Maxwell–Boltzmann distribution function.

Combine with BTE in the case of no external force, we have:
$$
\frac{\partial f}{\partial t} + c\cdot \nabla f = \frac{1}{\tau}(f^{eq}-f)
$$

Assumed it is valid along specific direction. Then we have:
$$
\frac{\partial f_i}{\partial t} + c_i\cdot \nabla f_i = \frac{1}{\tau}(f_i^{eq}-f_i)
$$

Comment on this formula:
1. The equation is a linear partial differential equation.
2. The equation looks like an advection equation with a source term.
3. The right-hand side of the equation represents the advection (streaming).
4. The left-hand side term represents the collision process, source term.

Discretized it
$$
f_i(r+c_i\Delta t, t + \Delta t) = f_i(r, t) + \frac{\Delta t}{\tau}[f_i^{eq}(r, t) - f_i(r, t)]
$$

## Lattice Arrangement - DnQm
Notice that max number of factitious particles at most m.

### One-Dimension: Assume $\Delta x = \Delta t$
- D1Q3: $f_0$ is center, $f_1$ is one grid toward left, $f_2$ is one grid toward right  
    - weighting factors $\omega_i$ are $\frac{4}{6},\frac{1}{6},\frac{1}{6}$ for $f_0, f_1, f_2$
    - Sound speed $c_s = \frac{1}{\sqrt{3}}$

### Two-Dimension: Assume $\Delta x = \Delta y = \Delta t$
- D2Q9: $f_0$ is center, $f_1$ to $f_4$ are grids one step away, $f_5$ to $f_8$ are grids two step away (diagonal)
    - weighting factors $\omega_i$ are $\frac{4}{9},\frac{1}{9},\frac{1}{36}$ for $f_0, f_1 - f_4, f_5 - f_8$
    
### Three-Dimension: Assume $\Delta x = \Delta y = \Delta z = \Delta t$
- D3Q15/D3Q19: check textbook

## Equilibrium Distribution Function

For particles moving in a medium with macroscopic velocity $u$. The normalized Maxwell’s distribution function can be written as
$$
f = \frac{\rho}{2\pi/3}e^{-\frac{3}{2}(c-u)^2} = \frac{\rho}{2\pi/3}e^{-\frac{3}{2}c^2}e^{3(c\cdot u- u^2)/2}
$$

recall Tayler Expansion for $e^{-x}$ is 
$$
e^{-x} = \sum_{n = 0}^\infty \frac{(-x)^n}{n!}
$$

Hence we can rewrite $f$ as
$$
f = \frac{\rho}{2\pi/3}e^{-\frac{3}{2}(c-u)^2} = \frac{\rho}{2\pi/3}e^{-\frac{3}{2}c^2}[1 + 3(c\cdot u) - \frac{3}{2}u^2 + \dots]
$$

The general form of the equilibrium distribution function is
$$
f_i^{eq} = \Phi\omega_i[A + B(c_i\cdot u) + C(c_i\cdot u)^2 + D u^2]
$$
where $u$ is the macroscopic flow velocity vector; $A, B, C, D$ are constants that determined based on conservation principle (mass, momentum, energy). $\Phi$ is scalar parameter such as density ($\rho$), temperature (thermal energy density), or speices concentration.
$$
\Phi = \sum_{i=0}^{i=n}f_i^{eq}
$$
where $n$ is number of lattice links (the m in DnQm)